## データ構造操作ベンチマークの分析

データ構造操作（リスト、ハッシュ、セット、ソート済みセットなど）のベンチマーク結果を分析します。

In [ ]:
def analyze_ds_benchmark(redis_result, valkey_result):
    if not redis_result or not valkey_result:
        print("データ構造操作ベンチマークの結果が見つかりません。")
        return
    
    # 設定情報を表示
    redis_config = redis_result['config']
    valkey_config = valkey_result['config']
    
    print("ベンチマーク設定:")
    print(f"データ構造: {redis_config['data_structure']}")
    print(f"要素サイズ: {redis_config['element_size']} バイト")
    print(f"キー数: {redis_config['num_keys']}")
    print(f"キーあたりの要素数: {redis_config['elements_per_key']}")
    print(f"スレッド数: {redis_config['num_threads']}")
    print(f"スレッドあたりの操作数: {redis_config['operations_per_thread']}")
    print(f"読み取り/書き込み比率: {redis_config['read_write_ratio']}")
    
    # 基本的な統計情報を抽出
    redis_stats = redis_result['stats']
    valkey_stats = valkey_result['stats']
    
    # 比較データフレームを作成
    comparison_data = {
        'メトリクス': [
            'スループット (ops/sec)',
            '平均レイテンシ (ms)',
            '中央値レイテンシ (ms)',
            'p95レイテンシ (ms)',
            'p99レイテンシ (ms)'
        ],
        'Redis': [
            redis_stats['operations_per_sec'],
            redis_stats['all_stats']['avg'],
            redis_stats['all_stats']['median'],
            redis_stats['all_stats']['p95'],
            redis_stats['all_stats']['p99']
        ],
        'Valkey': [
            valkey_stats['operations_per_sec'],
            valkey_stats['all_stats']['avg'],
            valkey_stats['all_stats']['median'],
            valkey_stats['all_stats']['p95'],
            valkey_stats['all_stats']['p99']
        ]
    }
    
    # 差異を計算
    comparison_data['差異 (%)'] = [
        (valkey_stats['operations_per_sec'] - redis_stats['operations_per_sec']) / redis_stats['operations_per_sec'] * 100,
        (redis_stats['all_stats']['avg'] - valkey_stats['all_stats']['avg']) / redis_stats['all_stats']['avg'] * 100,
        (redis_stats['all_stats']['median'] - valkey_stats['all_stats']['median']) / redis_stats['all_stats']['median'] * 100,
        (redis_stats['all_stats']['p95'] - valkey_stats['all_stats']['p95']) / redis_stats['all_stats']['p95'] * 100,
        (redis_stats['all_stats']['p99'] - valkey_stats['all_stats']['p99']) / redis_stats['all_stats']['p99'] * 100
    ]
    
    # データフレームを作成
    df = pd.DataFrame(comparison_data)
    
    # 結果を表示
    print("\n全体の比較:")
    display(df)
    
    # データ構造別の比較
    if 'structure_stats' in redis_stats and 'structure_stats' in valkey_stats:
        # 共通のデータ構造を特定
        redis_structures = set(redis_stats['structure_stats'].keys())
        valkey_structures = set(valkey_stats['structure_stats'].keys())
        common_structures = redis_structures.intersection(valkey_structures)
        
        for ds_type in common_structures:
            redis_ds_stats = redis_stats['structure_stats'][ds_type]
            valkey_ds_stats = valkey_stats['structure_stats'][ds_type]
            
            # データ構造別の比較データフレームを作成
            ds_comparison = {
                'メトリクス': [
                    'スループット (ops/sec)',
                    '平均レイテンシ (ms)',
                    'p95レイテンシ (ms)'
                ],
                'Redis': [
                    redis_ds_stats['throughput'],
                    redis_ds_stats['avg'],
                    redis_ds_stats['p95']
                ],
                'Valkey': [
                    valkey_ds_stats['throughput'],
                    valkey_ds_stats['avg'],
                    valkey_ds_stats['p95']
                ]
            }
            
            # 差異を計算
            ds_comparison['差異 (%)'] = [
                (valkey_ds_stats['throughput'] - redis_ds_stats['throughput']) / redis_ds_stats['throughput'] * 100,
                (redis_ds_stats['avg'] - valkey_ds_stats['avg']) / redis_ds_stats['avg'] * 100,
                (redis_ds_stats['p95'] - valkey_ds_stats['p95']) / redis_ds_stats['p95'] * 100
            ]
            
            # データフレームを作成
            ds_df = pd.DataFrame(ds_comparison)
            
            # 結果を表示
            print(f"\n{ds_type.upper()} データ構造の比較:")
            display(ds_df)
            
            # 操作タイプ別の比較（データ構造の操作タイプ別）
            if 'operations_by_type' in redis_ds_stats and 'operations_by_type' in valkey_ds_stats:
                redis_op_types = set(redis_ds_stats['operations_by_type'].keys())
                valkey_op_types = set(valkey_ds_stats['operations_by_type'].keys())
                common_op_types = redis_op_types.intersection(valkey_op_types)
                
                for op_type in common_op_types:
                    redis_op_stats = redis_ds_stats['operations_by_type'][op_type]
                    valkey_op_stats = valkey_ds_stats['operations_by_type'][op_type]
                    
                    # 操作タイプ別の比較データフレームを作成
                    op_comparison = {
                        'メトリクス': [
                            'スループット (ops/sec)',
                            '平均レイテンシ (ms)',
                            'p95レイテンシ (ms)'
                        ],
                        'Redis': [
                            redis_op_stats['throughput'],
                            redis_op_stats['avg'],
                            redis_op_stats['p95']
                        ],
                        'Valkey': [
                            valkey_op_stats['throughput'],
                            valkey_op_stats['avg'],
                            valkey_op_stats['p95']
                        ]
                    }
                    
                    # 差異を計算
                    op_comparison['差異 (%)'] = [
                        (valkey_op_stats['throughput'] - redis_op_stats['throughput']) / redis_op_stats['throughput'] * 100,
                        (redis_op_stats['avg'] - valkey_op_stats['avg']) / redis_op_stats['avg'] * 100,
                        (redis_op_stats['p95'] - valkey_op_stats['p95']) / redis_op_stats['p95'] * 100
                    ]
                    
                    # データフレームを作成
                    op_df = pd.DataFrame(op_comparison)
                    
                    # 結果を表示
                    print(f"\n{ds_type.upper()} - {op_type} 操作の比較:")
                    display(op_df)
        
        # データ構造別のグラフ作成
        if common_structures:
            # スループット比較グラフ
            throughput_data = {
                'データ構造': [],
                'エンジン': [],
                'スループット (ops/sec)': []
            }
            
            for ds_type in common_structures:
                throughput_data['データ構造'].extend([ds_type.upper(), ds_type.upper()])
                throughput_data['エンジン'].extend(['Redis', 'Valkey'])
                throughput_data['スループット (ops/sec)'].extend([
                    redis_stats['structure_stats'][ds_type]['throughput'],
                    valkey_stats['structure_stats'][ds_type]['throughput']
                ])
            
            throughput_df = pd.DataFrame(throughput_data)
            
            plt.figure(figsize=(14, 8))
            sns.barplot(x='データ構造', y='スループット (ops/sec)', hue='エンジン', data=throughput_df)
            plt.title('データ構造別スループット比較')
            plt.ylabel('操作/秒')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            
            # レイテンシ比較グラフ
            latency_data = {
                'データ構造': [],
                'エンジン': [],
                'レイテンシ (ms)': []
            }
            
            for ds_type in common_structures:
                latency_data['データ構造'].extend([ds_type.upper(), ds_type.upper()])
                latency_data['エンジン'].extend(['Redis', 'Valkey'])
                latency_data['レイテンシ (ms)'].extend([
                    redis_stats['structure_stats'][ds_type]['avg'],
                    valkey_stats['structure_stats'][ds_type]['avg']
                ])
            
            latency_df = pd.DataFrame(latency_data)
            
            plt.figure(figsize=(14, 8))
            sns.barplot(x='データ構造', y='レイテンシ (ms)', hue='エンジン', data=latency_df)
            plt.title('データ構造別平均レイテンシ比較')
            plt.ylabel('ミリ秒')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
    
    # 全体のグラフ作成
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # スループットの比較
    throughput_data = {
        'エンジン': ['Redis', 'Valkey'],
        'スループット (ops/sec)': [redis_stats['operations_per_sec'], valkey_stats['operations_per_sec']]
    }
    throughput_df = pd.DataFrame(throughput_data)
    
    sns.barplot(x='エンジン', y='スループット (ops/sec)', data=throughput_df, ax=axes[0])
    axes[0].set_title('全体スループット比較')
    axes[0].set_ylabel('操作/秒')
    
    # レイテンシの比較
    latency_data = {
        'エンジン': ['Redis', 'Redis', 'Redis', 'Valkey', 'Valkey', 'Valkey'],
        'メトリクス': ['平均', 'p95', 'p99', '平均', 'p95', 'p99'],
        'レイテンシ (ms)': [
            redis_stats['all_stats']['avg'],
            redis_stats['all_stats']['p95'],
            redis_stats['all_stats']['p99'],
            valkey_stats['all_stats']['avg'],
            valkey_stats['all_stats']['p95'],
            valkey_stats['all_stats']['p99']
        ]
    }
    latency_df = pd.DataFrame(latency_data)
    
    sns.barplot(x='メトリクス', y='レイテンシ (ms)', hue='エンジン', data=latency_df, ax=axes[1])
    axes[1].set_title('全体レイテンシ比較')
    axes[1].set_ylabel('ミリ秒')
    
    plt.tight_layout()
    plt.show()

# データ構造操作ベンチマークの分析を実行
if ds_redis_result and ds_valkey_result:
    analyze_ds_benchmark(ds_redis_result, ds_valkey_result)
else:
    print("データ構造操作ベンチマークの結果が見つかりません。")

## 結論

ベンチマークテストの結果から、RedisとValkeyの性能差について以下の結論が導き出せます：

1. **スループット**：全体的なスループット（操作/秒）において、どちらのエンジンが優れているかを確認できます。
2. **レイテンシ**：平均、p95、p99のレイテンシにおいて、どちらのエンジンが低いレイテンシを示しているかを確認できます。
3. **データ構造別の性能**：各データ構造（文字列、リスト、ハッシュ、セット、ソート済みセット）において、どちらのエンジンが優れているかを確認できます。
4. **操作タイプ別の性能**：読み取り操作（GET）と書き込み操作（SET）において、どちらのエンジンが優れているかを確認できます。

これらの結果を総合的に分析することで、特定のユースケースに最適なエンジンを選択するための判断材料となります。例えば、読み取り操作が多いアプリケーションでは読み取り性能が優れているエンジンを、書き込み操作が多いアプリケーションでは書き込み性能が優れているエンジンを選択するなど、ワークロードに応じた最適な選択が可能になります。